In [49]:
!pip install pandas langchain neo4j transformers tokenizers torch

   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/199.4 MB ? eta -:--:--
   ---------------------------------------- 1.0/199.4 MB 3.9 MB/s eta 0:00:52
   ---------------------------------------- 2.1/199.4 MB 4.5 MB/s eta 0:00:44
    --------------------------------------- 3.7/199.4 MB 5.3 MB/s eta 0:00:37
    --------------------------------------- 4.7/199.4 MB 5.4 MB/s eta 0:00:37
   - -------------------------------------- 6.8/199.4 MB 6.4 MB/s eta 0:00:31
   - -------------------------------------- 9.4/199.4 MB 7.3 MB/s eta 0:00:26
   -- ------------------------------------- 12.3/199.4 MB 8.3 MB/s eta 0:00:23
   --- ------------------------------------ 16.3/199.4 MB 9.6 MB/s eta 0:00:20
   ---- ----------------------------------- 21.0/199.4 MB 11.0 MB/s eta 0:00:17
   ---- ----------------------------------- 23.6/199.4 MB 11.5 MB/s eta 0:00:16
 

In [5]:
import sqlite3
import pandas as pd

In [16]:
# Read CSV file
# data from https://www.kaggle.com/datasets/everydaycodings/global-news-dataset?select=data.csv
df = pd.read_csv("data.csv")
df.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,full_content
0,89541,NaN,International Business Times,Paavan MATHEMA,UN Chief Urges World To 'Stop The Madness' Of ...,UN Secretary-General Antonio Guterres urged th...,https://www.ibtimes.com/un-chief-urges-world-s...,https://d.ibtimes.com/en/full/4496078/nepals-g...,2023-10-30 10:12:35.000000,UN Secretary-General Antonio Guterres urged th...,Nepal,UN Secretary-General Antonio Guterres urged th...
1,89542,NaN,Prtimes.jp,NaN,RANDEBOOよりワンランク上の大人っぽさが漂うニットとベストが新登場。,[株式会社Ainer]\nRANDEBOO（ランデブー）では2023年7月18日(火)より公...,https://prtimes.jp/main/html/rd/p/000000147.00...,https://prtimes.jp/i/32220/147/ogp/d32220-147-...,2023-10-06 04:40:02.000000,"RANDEBOO2023718()WEB2023 Autumn Winter \n""Nepa...",Nepal,NaN
2,89543,NaN,VOA News,webdesk@voanews.com (Agence France-Presse),UN Chief Urges World to 'Stop the Madness' of ...,UN Secretary-General Antonio Guterres urged th...,https://www.voanews.com/a/un-chief-urges-world...,https://gdb.voanews.com/01000000-0a00-0242-60f...,2023-10-30 10:53:30.000000,"Kathmandu, Nepal UN Secretary-General Antonio...",Nepal,NaN
3,89545,NaN,The Indian Express,Editorial,Sikkim warning: Hydroelectricity push must be ...,Ecologists caution against the adverse effects...,https://indianexpress.com/article/opinion/edit...,https://images.indianexpress.com/2023/10/edit-...,2023-10-06 01:20:24.000000,At least 14 persons lost their lives and more ...,Nepal,At least 14 persons lost their lives and more ...
4,89547,NaN,The Times of Israel,Jacob Magid,"200 foreigners, dual nationals cut down in Ham...","France lost 35 citizens, Thailand 33, US 31, U...",https://www.timesofisrael.com/200-foreigners-d...,https://static.timesofisrael.com/www/uploads/2...,2023-10-27 01:08:34.000000,"Scores of foreign citizens were killed, taken ...",Nepal,NaN


In [17]:
df.isnull().sum()

article_id          0
source_id       80880
source_name         0
author           8219
title              40
description       383
url                 0
url_to_image     5624
published_at        0
content             0
category           42
full_content    46943
dtype: int64

In [19]:
df = df.drop(columns=['source_id', 'author', 'url_to_image'])

In [22]:
df = df.dropna()

In [40]:
# Connect to SQLite
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Create database
df.to_sql('news_data', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

In [44]:
from neo4j import GraphDatabase

# Connect to SQLite
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Connect to Neo4j
uri = "bolt://localhost:7687"  # Adjust if needed
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))



In [50]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load LLama 2 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

# Function to generate text using Llama 2
def generate_llama2_output(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Sample use
prompt = "Extract entities and relationships from this text: 'Barack Obama was born in Hawaii.'"
print(generate_llama2_output(prompt))

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-hf.
401 Client Error. (Request ID: Root=1-66f9a717-34eebcb25dc2847e02cbde60;7a3b64e0-d549-4d46-8538-65ada59edbb1)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted. You must have access to it and be authenticated to access it. Please log in.

In [45]:
# NEEDS TO BE REVISED TO CREATE BASED ON THE GRAPH MADE BY LLM

# # Define the function to create nodes and relationships in Neo4j
# def create_article(tx, article_id, title, url, content, full_content, description, published_at, source_name, category):
#     query = """
#     MERGE (a:Article {article_id: $article_id, title: $title, url: $url, content: $content, full_content: $full_content, description: $description})
#     MERGE (s:Source {source_name: $source_name})
#     MERGE (c:Category {category: $category})
#     MERGE (d:Date {published_at: $published_at})
#     MERGE (a)-[:FROM_SOURCE]->(s)
#     MERGE (a)-[:BELONGS_TO]->(c)
#     MERGE (a)-[:PUBLISHED_ON]->(d)
#     """
#     tx.run(query, article_id=article_id, title=title, url=url, content=content, full_content=full_content, description=description, published_at=published_at, source_name=source_name, category=category)

# # Fetch the data from SQLite and insert it into Neo4j
# def load_data_to_neo4j():
#     with driver.session() as session:
#         cursor.execute("SELECT article_id, source_name, title, description, url, published_at, content, category, full_content FROM news_data")
#         rows = cursor.fetchall()
#         for row in rows:
#             session.execute_write(create_article, row[0], row[2], row[4], row[6], row[8], row[3], row[5], row[1], row[7])

# # Load the data
# load_data_to_neo4j()

# # Close connections
# driver.close()
# conn.close()

BufferError: Existing exports of data: object cannot be re-sized